# Small METR Dataset Generation

실제 크기로는 계산량이 많아 테스트가 힘드므로 작고 확실한 데이터셋을 만들어 테스트

In [ ]:
from metr.components.metr_imc.traffic_data import get_raw
from metr_val import PATH_CONF

raw = get_raw(PATH_CONF.dataset.metr_imc)
raw_df = raw.data
raw_df.head(3)

In [ ]:
from metr.components.metr_imc.outlier import RemovingWeirdZeroOutlierProcessor

processor = RemovingWeirdZeroOutlierProcessor()
processed_df = processor.process(raw_df)

processed_df.head(3)

In [ ]:
# 0 Outlier 제거 전 결측치 10% 이하 센서 수와 리스트
missing_ratio_before = raw_df.isnull().sum() / len(raw_df)
sensors_with_high_missing_before = missing_ratio_before[missing_ratio_before < 0.1]

print(f"0 Outlier 제거 전 결측치 10% 이하인 센서 수: {len(sensors_with_high_missing_before)}")
print(f"전체 센서 수: {len(raw_df.columns)}")
print(f"비율: {len(sensors_with_high_missing_before) / len(raw_df.columns) * 100:.2f}%")
print(f"\n결측치 10% 이하인 센서 리스트:")
print(sensors_with_high_missing_before.index.tolist())

In [ ]:
# 0 Outlier 제거 후 결측치 10% 이하 센서 수와 리스트
missing_ratio_after = processed_df.isnull().sum() / len(processed_df)
sensors_with_high_missing_after = missing_ratio_after[missing_ratio_after < 0.1]

print(f"0 Outlier 제거 후 결측치 10% 이하인 센서 수: {len(sensors_with_high_missing_after)}")
print(f"전체 센서 수: {len(processed_df.columns)}")
print(f"비율: {len(sensors_with_high_missing_after) / len(processed_df.columns) * 100:.2f}%")
print(f"\n결측치 10% 이하인 센서 리스트:")
print(sensors_with_high_missing_after.index.tolist())

In [ ]:
import matplotlib.pyplot as plt

# processed_df의 결측치 수 히스토그램
missing_counts = processed_df[sensors_with_high_missing_after.index].isnull().sum()

plt.figure(figsize=(12, 6))
plt.hist(missing_counts, bins=50, edgecolor="black", alpha=0.7)
plt.title("Histogram of Missing Values Count per Sensor (After Outlier Removal)")
plt.xlabel("Number of Missing Values")
plt.ylabel("Number of Sensors")
plt.grid(True, alpha=0.3)
plt.show()

print(f"결측치 수 통계:")
print(f"원래 전체 데이터 수: {len(processed_df)}")
print(f"평균: {missing_counts.mean():.2f}")
print(f"중간값: {missing_counts.median():.2f}")
print(f"최소값: {missing_counts.min()}")
print(f"최대값: {missing_counts.max()}")
print(f"표준편차: {missing_counts.std():.2f}")

In [ ]:
import geopandas as gpd

nodelink_raw = gpd.read_file("../datasets/metr-imc/nodelink/imc_link.shp")
nodelink_raw.head(3)

In [ ]:
nodelink_raw.explore()

In [ ]:
# 10% 이하 결측치를 가진 센서를 nodelink_raw에서 추출하여 지도에 표시
sensors_link_ids = sensors_with_high_missing_after.index.tolist()

# LINK_ID가 센서 ID와 일치하는 링크만 필터링
filtered_nodelink = nodelink_raw[nodelink_raw['LINK_ID'].isin(sensors_link_ids)]

print(f"10% 이하 결측치를 가진 센서 수: {len(sensors_link_ids)}")
print(f"nodelink_raw에서 찾은 해당 센서 수: {len(filtered_nodelink)}")
print(f"일치율: {len(filtered_nodelink) / len(sensors_link_ids) * 100:.2f}%")

# 지도에 표시
filtered_nodelink.explore()

In [ ]:
save_folder_path = "../traffic_imc/data"
save_file_path = f"{save_folder_path}/less10mv_dataset.shp"

In [ ]:
filtered_nodelink.to_file(save_file_path, driver='ESRI Shapefile')

## QGIS에서 편집한 Shapefile

In [ ]:
selected_small_gdf = gpd.read_file(save_folder_path + "/selected_small_dataset.shp")
selected_small_gdf.explore()

In [ ]:
selected_sensors_gdf = selected_small_gdf['LINK_ID']
selected_sensors_list = selected_sensors_gdf.unique().tolist()

print(f"선택된 센서 수(Not Unique): {len(selected_sensors_gdf)}")
print(f"선택된 센서 수: {len(selected_sensors_list)}")

In [ ]:
from metr.pipeline import generate_subset_dataset
import os

save_selected_dir_path = f"{save_folder_path}/selected_small_v1"
os.makedirs(save_selected_dir_path, exist_ok=True)

generate_subset_dataset(
    target_nodelink=selected_sensors_list,
    save_dir_path=save_selected_dir_path,
)

In [ ]:
from metr.components.metr_imc.traffic_data import get_raw
from metr.pipeline import PATH_CONF

traffic_save_path = os.path.join(
    save_selected_dir_path,
    PATH_CONF.raw["dataset"]["filenames"]["metr_imc"],
)

raw = get_raw(traffic_save_path)
raw_df = raw.data
raw_df.head(3)

In [ ]:
from metr.pipeline import generate_distances_shapefile

distance_file_path = os.path.join(
    save_selected_dir_path,
    PATH_CONF.raw["dataset"]["filenames"]["distances"],
)
sensor_locations_path = PATH_CONF.sensor_locations_path
output_dir_path = os.path.join(save_selected_dir_path, "misc")
os.makedirs(output_dir_path, exist_ok=True)
output_path = os.path.join(output_dir_path, "distances.shp")

generate_distances_shapefile(
    distances_path=distance_file_path,
    sensor_locations_path=sensor_locations_path,
    output_path=output_path,
)

In [ ]:
distance_gdf = gpd.read_file(output_path)
distance_gdf.explore()